# Into to NNCME with a Gene Expression Example - Detailed Version

This is a detailed step-by-step guideline of implementing a gene expression example with NNCME. \
A simplified version can be seen in `Start with a Simple Gene Expression Example.ipynb`.\
If you would like to implement a more complex system, please refer to the toggle switch example (`Toggle Switch System.ipynb`).\
You can also explore more through the `.py` files of other representative examples.

## 1. Gene Expression System

The gene expression system involves two species: mRNA ($r$) , protein ($p$).

<table>
<td> 
<img src="https://github.com/jiadeyu0602/CheatSheet/raw/master/GE_panela.svg" width="120"/> <br>
</td> 
</table>

The model can be written as the chemical reactions:
\begin{align}
\begin{split}
DNA\stackrel{k_{r}}{\longrightarrow}r,\quad
r\stackrel{k_{p}}{\longrightarrow}r+p,\quad
r\stackrel{\gamma_{r}}{\longrightarrow}\emptyset,\quad
p\stackrel{\gamma_{p}}{\longrightarrow}\emptyset, 
\end{split}
\end{align}
where $k_r$, $k_p$, $\gamma_r$ and $\gamma_p$ are rate constants.

The concentrations of mRNA and protein are given by the following ODEs: 

\begin{align}
\frac{dr}{dt}=k_r-\gamma_{r}r\\
\frac{dp}{dt}=k_{p}r-\gamma_{p}p
\end{align}

The stoichiometric matrix of this chemical reaction is written as

$$V=
\begin{bmatrix} 
    1 & 0 & -1 & 0 \\ 
    0 & 1 & 0 & -1 
\end{bmatrix}.
$$

## 2. Create a GeneExp.py
Input gene expression system as a `.py` file. Include initial conditions, species number constraint, reaction rates and the stoichiometric matrix in the function 'rates' . What's more, function 'init' and 'Propensity' are almost the same and are essential for every system.

In [ ]:
import numpy as np
import torch

class GeneExp:
    
    def rates(self):  
        
        self.L=2
        IniDistri='delta'
        initialD=np.array([0,0]).reshape(1,self.L) # the parameter for the initial delta distribution
        r=torch.zeros(4) #Reaction rates
        r[0] = 0.1 #kr
        r[1] = 0.1 #kp
        r[2] = 0.1 #yr
        r[3] = 0.002 #yp
        
        # Stoichiometric matrix = ReactionMatRight - ReactionMatLeft #SpeciesXReactions    
        ReactionMatLeft=torch.as_tensor([(0, 1,1,0), (0,0,0,1)]).to(self.device)#SpeciesXReactions
        ReactionMatRight=torch.as_tensor([(1, 1,0,0), (0,1,0,0)]).to(self.device)#SpeciesXReactions

        MConstrain=np.zeros(1,dtype=int)
        conservation=np.ones(1,dtype=int)
        
        return IniDistri,initialD,r,ReactionMatLeft,ReactionMatRight,MConstrain,conservation
    
    def __init__(self, *args, **kwargs):
        super().__init__()
        #self.n = kwargs['n']
        self.L = kwargs['L']
        self.M = kwargs['M']
        self.bits = kwargs['bits']  
        self.device = kwargs['device']
        self.MConstrain = kwargs['MConstrain']
        self.Para = kwargs['Para']
        self.IniDistri = kwargs['IniDistri']
        self.binary = kwargs['binary']
        self.order = kwargs['order']
        
    def Propensity(self,Win,Wout,cc,SampleNeighbor1D_Win,SampleNeighbor1D,NotHappen_in_low,NotHappen_in_up,NotHappen_out_low,NotHappen_out_up):
        Propensity_in=torch.prod(Win,1)*cc#torch.tensor(r, dtype=torch.float64).to(self.device)   
        Propensity_out=torch.prod(Wout,1)*cc#torch.tensor(r, dtype=torch.float64).to(self.device)    
    
        return Propensity_in,Propensity_out

## 3. Implement the code after providing the GeneExp.py file

(a) Server: Use a `GeneExp.sh` to input the hyperparameters from the shell. 

In [ ]:
#!/bin/bash
#SBATCH --job-name=GeneExp-rnn-1-16-M100-Tstep1001-dt0.1
#SBATCH --ntasks=1
#SBATCH --nodes=1
#SBATCH --partition=v100
#SBATCH --gres=gpu:1
#SBATCH -o out/%j.out
#SBATCH -e out/%j.out
#SBATCH -t 13-2:00:00

# load the environment
module purge

CUDA_LAUNCH_BLOCKING=1 python3 MasterEq.py --L 2 --M 100 --Model 'GeneExp' --net 'rnn' --lossType 'kl' --max_stepAll 5000 --max_stepLater 100 --lr 0.001 --net_depth 1 --net_width 16 --print_step 20 --batch_size 1000 --Tstep 1001 --delta_t 0.1 --cuda 0 --dtype float64

#exit 0


(b) PC users (Windows): Run `MasterEq.py` after necessary changes according to the gene expression example (importing the model class, adjusting the hyperparameters and adding model command).

In [ ]:
###MasterEq.py
from args import args
import numpy as np
from main import Test

###Add models----------------------------------
from ToggleSwitch import ToggleSwitch  
from EarlyLife import EarlyLife
from Epidemic import Epidemic
from cascade1 import cascade1
from cascade1_inverse import cascade1_inverse
from cascade2 import cascade2
from cascade3 import cascade3
from BirthDeath import BirthDeath
from GeneExp import GeneExp #from GeneExp.py file import the GeneExp class
from AFL import AFL

##Set parameters-------------------------------
###Initialize parameters: otherwise the parameters are specified in init_out_dir-> args.py
args.Model='GeneExp' #Model name
args.L=2#Species number
args.M=int(100) #Upper limit of the molecule number: it is adjustable and can be indicated by doing a few Gillespie simulation. 
args.batch_size=1000 #Number of batch samples
args.Tstep=1001# Time step of iterating the chemical master equation
args.delta_t=0.1 #Time step length of iterating the chemical master equation, depending on the reaction rates

args.net ='rnn'
args.max_stepAll=5000 #Maximum number of steps first time step (usually larger to ensure the accuracy)
args.max_stepLater=100 #Maximum number of steps of later time steps
args.net_depth=1 # including output layer and not input data layer
args.net_width=16
args.d_model=16# transformer
args.d_ff=32# transformer
args.n_layers=2# transformer
args.n_heads=2# transformer
args.lr=0.001
args.binary=False
args.AdaptiveT=False
args.AdaptiveTFold=5
args.print_step=20
args.saving_data_time_step=[0,1e2,5e2,2e3,1e4,2e4,5e4,1e5,1.5e5,2e5,2.5e5,3e5,3.5e5,4e5,5e5,6e5,7e5,8e5,9e5,1e6] #To save data at which time steps (give in a list)
args.training_loss_print_step=[0,1,2,101,1001,2e3,1e4,1e5,2e5,3e5,4e5,5e5] #To print training loss at which time steps (give in a list)

###Default parameters:
args.bias=True
args.bits=1
if args.binary:
    args.bits=int(np.ceil(np.log2(args.M)))
args.Percent=0.2         
args.clip_grad=1
args.dtype='float64'
args.epsilon=1e-30#initial probability of zero species number
args.lr_schedule=False#True

###Add model command-------------------------------
if args.Model=='ToggleSwitch':
    model = ToggleSwitch(**vars(args))   
if args.Model=='EarlyLife':
    model = EarlyLife(**vars(args))   
if args.Model=='Epidemic':
    model = Epidemic(**vars(args))   
if args.Model=='cascade1': 
    model = cascade1(**vars(args)) 
if args.Model=='cascade1_inverse':
    model = cascade1_inverse(**vars(args)) 
if args.Model=='cascade2':
    model = cascade2(**vars(args))    
if args.Model=='cascade3':
    model = cascade3(**vars(args))    
if args.Model=='BirthDeath':
    model = BirthDeath(**vars(args))   
if args.Model=='GeneExp': 
    model = GeneExp(**vars(args))   
if args.Model=='AFL':
    model = AFL(**vars(args)) 
    
#Run model-----------------------------------------        
if __name__ == '__main__':
    Test(model)    


## 4. Optional: Run Gillespie simulation
Gillespie algorithm, also known as the stochastic simulation algorithm or the kinetic Monte Carlo, simulates trajectories to generate statistics of relevant variables. To evaluate the accuracy of the learnt distribution by the VAN, we can compare the resultant marginal distribution of one species with those from Gillespie algorithm.

In [ ]:
import numpy as np
import biocircuits
import matplotlib.pyplot as plt

def GeneExp_propensity(
    propensities, population, t, kr, kp, yr, yp
):
    #species
    r, p = population
    propensities[0] = kr
    propensities[1] = kp*r
    propensities[2] = yr*r
    propensities[3] = yp*p

#the stoichiometric matrix       
GeneExp_update = np.array(
    [
        [ 1, 0 ], 
        [ 0, 1 ],
        [-1, 0 ],
        [ 0,-1 ], 
    ],dtype=int)

#the reaction rates 
kr = 0.1 #kr
kp = 0.1 #kp
yr = 0.1 #yr
yp = 0.002 #yp 

GeneExp_args = (kr, kp, yr, yp)

#initial number of species 
r0=0
p0=0
GeneExp_pop_0 = np.array([r0,p0], dtype=float) # follow VAN's learnt initial number

#simulation time length
T=3600
time_points = np.linspace(0, T, int(T/60))

In [ ]:
Run=1 #run Gillespie or not
times=1000 #Gillespie simulation times

out_filename = 'GeneExp_times'+str(times)+'_T'+str(T)+'_dis'+str(r0)+'_'+str(p0) #filename to save
if Run==1:
    
    r_total=np.empty(shape=(0,len(time_points)))#to save the time evolution of mRNA number in each simulation (dimension: times*time_points)
    p_total=np.empty(shape=(0,len(time_points)))#to save the time evolution of protein number in each simulation (dimension: times*time_points)
    
    for i in range(times):
        
        # Perform the Gillespie simulation
        pop = biocircuits.gillespie_ssa(
            GeneExp_propensity,
            GeneExp_update,
            GeneExp_pop_0,
            time_points,
            args=GeneExp_args,
        )
        

        r_total=np.row_stack((r_total,pop[0,:,0]))
        p_total=np.row_stack((p_total,pop[0,:,1]))


        if i<3: #Plot fist three simulations
            plt.figure()
            plt.plot(time_points ,pop[0,:,0])     
            plt.xlabel("Time")
            plt.ylabel("mRNA number")
            plt.grid()
            plt.show()
            
            plt.figure()
            plt.plot(time_points ,pop[0,:,1])          
            plt.xlabel("Time")
            plt.ylabel("Protein number")
            plt.grid()
            plt.show()
        
    np.savez('{}'.format(out_filename),np.array(times),np.array(time_points),r_total,p_total) #sava data    
else: #load existing data file
    data=np.load(out_filename+'.npz')
    print(list(data))    
    time_points = data['arr_1']
    r_total = data['arr_2']
    p_total = data['arr_3']

In [ ]:
#Plot Gillespie Result
plt.rc('font', size=16)
plt.figure(num=None,  dpi=400, edgecolor='k')
fig, axes = plt.subplots(2,1)
fig.tight_layout()
ax = plt.subplot(1,1, 1)
plt.errorbar(time_points, np.mean(r_total,0),  yerr=np.std(r_total,0))    
plt.xlabel('Time')
plt.ylabel('Species number')
plt.legend()
fig.set_size_inches(9, 8)
plt.figure()

plt.rc('font', size=16)
plt.figure(num=None,  dpi=400, edgecolor='k')
fig, axes = plt.subplots(2,1)
fig.tight_layout()
ax = plt.subplot(1,1, 1)
plt.errorbar(time_points, np.mean(p_total,0),  yerr=np.std(p_total,0))    
plt.xlabel('Time')
plt.ylabel('Species number')
plt.legend()
fig.set_size_inches(9, 8)
plt.figure()

## 5. Plot Results
Plot the result of the gene expression. Relevant details can be referred in the Supplement Information of the manuscript.

In [ ]:
# load data of the VAN and Gillespie
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import pandas as pd
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import matplotlib as mpl

jet = cm.get_cmap('jet')
jet_12_colors = jet(np.linspace(0, 1, 15))
plt.rc('font', size=48)

path1="GeneExpression\GeneExp1_times10000_T3600_dis0_0.npz" # Gillespie data path
path2="GeneExpression\Data_GeneExp1_M100_T36001_dt0.1_batch1000.npz" # VAN data path

data1=np.load(path1)
times = data1['arr_0']
time_points = data1['arr_1']
tfinal = time_points[-1]
step = time_points[1]-time_points[0]

rna_total = data1['arr_2']
prot_total = data1['arr_3']
    
rna_total_mean=np.mean(rna_total,0)
rna_total_std=np.std(rna_total,0)
prot_total_mean=np.mean(prot_total,0)
prot_total_std=np.std(prot_total,0)

data2=np.load(path2, allow_pickle=True)
argsSave = data2['arr_1']
delta_t=argsSave[1]
print_step= argsSave[7]
SampleSum=data2['arr_5']
delta_T= data2['arr_6']

AdaptiveT=0
if AdaptiveT: TimePoins=np.cumsum(delta_T)[np.arange(SampleSum.shape[0])*print_step]
else: TimePoins=np.cumsum(delta_T)[np.arange(SampleSum.shape[0])*print_step]*delta_t

### 5.1 The time evolution of the average counts of species

In [ ]:
# The time evolution of the average counts for the genes and proteins,  from the VAN (dots) and the Gillespie simulation (lines). 
#####curve-----------------------------------

markersize0=12

plt.figure(num=None,  dpi=400, edgecolor='k', linewidth=8)
fig, axes = plt.subplots(1,1)
plt.plot(time_points/3600,rna_total_mean,linewidth=5,color=jet_12_colors[3,:])
plt.plot(TimePoins[0:-1:13]/3600,np.mean(SampleSum[:,:,0],axis=1)[0:-1:13],
          marker='o',linestyle = 'None',color=jet_12_colors[3,:],markersize=markersize0)
plt.xlabel("Time (hr)")
plt.ylabel("mRNA")
plt.ylim(top=1.2)
fig.set_size_inches(9, 8)
plt.title('Average Count',fontsize=56)
plt.savefig('GE_panela1.svg', bbox_inches="tight", dpi=400)

plt.figure(num=None,  dpi=400, edgecolor='k', linewidth=8)
fig, axes = plt.subplots(1,1)
plt.plot(time_points/3600,prot_total_mean,linewidth=5,color=jet_12_colors[12,:])
plt.plot(TimePoins[0:-1:13]/3600,np.mean(SampleSum[:,:,1],axis=1)[0:-1:13],
          marker='o',linestyle = 'None',color=jet_12_colors[12,:],markersize=markersize0)
plt.xlabel("Time (hr)")
plt.ylabel("Protein")
plt.title('Average Count',fontsize=56)
plt.ylim(top=55)
fig.set_size_inches(9, 8)
plt.savefig('GE_panela2.svg', bbox_inches="tight", dpi=400)

<table>
<td> 
<img src="https://github.com/jiadeyu0602/CheatSheet/raw/master/GE_panela1.svg" width="200"/> <br>
</td> 
<td> 
<img src="https://github.com/jiadeyu0602/CheatSheet/raw/master/GE_panela2.svg" width="200"/> <br>
</td> 
</table>

### 5.2 Comparison of the means and standard deviations of the counts of species

In [ ]:
# Comparison of the means and standard deviations of the counts of the mRNA and protein between the VAN and the Gillespie simulation, at time points t=0, 0.004, 0.008, ..., 0.196, 0.2.

#####mean------------------------------------
markersize0=20
T=0.2
NumTimePoints=0.004
transparency=1-np.linspace(0, 0.99, int(T/NumTimePoints))
#mRNA
plt.figure(num=None,  dpi=400, edgecolor='k', linewidth=8)
fig, axes = plt.subplots(1,1)
Range=2
k=0
for tt in np.arange(0,T,NumTimePoints):
    if tt==0:
        plt.plot(np.mean(rna_total[:,int(tt*3600/step)],0),np.mean(SampleSum[:,:,0],axis=1)[round(tt*3600/np.max(TimePoins)*TimePoins.shape[0])],
          alpha=transparency[k], marker='o',linestyle = 'None',color=jet_12_colors[3,:],markersize=markersize0,label='mRNA')
    else:
        plt.plot(np.mean(rna_total[:,int(tt*3600/step)],0),np.mean(SampleSum[:,:,0],axis=1)[round(tt*3600/np.max(TimePoins)*TimePoins.shape[0])],
                  alpha=transparency[k], marker='o',linestyle = 'None',color=jet_12_colors[3,:],markersize=markersize0)
    k=k+1

plt.plot(np.array([0,Range]),np.array([0,Range]),linewidth=4,color='black')
plt.xlabel('Gillespie')
plt.ylabel('VAN')
plt.title('Mean')
plt.legend(fontsize=48,loc='best',handletextpad=0.2)
fig.set_size_inches(9, 8)
plt.savefig('GE_panelb11.svg', bbox_inches="tight", dpi=400)

#Protein
plt.figure(num=None,  dpi=400, edgecolor='k', linewidth=8)
fig, axes = plt.subplots(1,1)
Range=50
k=0
for tt in np.arange(0,T,NumTimePoints):
    if tt==0:
        plt.plot(np.mean(prot_total[:,int(tt*3600/step)],0),np.mean(SampleSum[:,:,1],axis=1)[round(tt*3600/np.max(TimePoins)*TimePoins.shape[0])],
                  alpha=transparency[k], marker='o',linestyle = 'None',color=jet_12_colors[12,:],markersize=markersize0,label='Protein')
    else:
        plt.plot(np.mean(prot_total[:,int(tt*3600/step)],0),np.mean(SampleSum[:,:,1],axis=1)[round(tt*3600/np.max(TimePoins)*TimePoins.shape[0])],
                  alpha=transparency[k], marker='o',linestyle = 'None',color=jet_12_colors[12,:],markersize=markersize0)
    k=k+1
plt.plot(np.array([0,Range]),np.array([0,Range]),linewidth=4,color='black')
plt.xlabel('Gillespie')
plt.ylabel('VAN')
plt.title('Mean')
plt.legend(fontsize=48,loc='best',handletextpad=0.2)
fig.set_size_inches(9, 8)
plt.savefig('GE_panelb12.svg', bbox_inches="tight", dpi=400)

#####std----------------------------------
#mRNA
plt.figure(num=None,  dpi=400, edgecolor='k', linewidth=8)
fig, axes = plt.subplots(1,1)
Range=2
k=0
for tt in np.arange(0,T,NumTimePoints):
    if tt==0:
        plt.plot(np.std(rna_total[:,int(tt*3600/step)],0),np.std(SampleSum[:,:,0],axis=1)[round(tt*3600/np.max(TimePoins)*TimePoins.shape[0])],
                  alpha=transparency[k], marker='o',linestyle = 'None',color=jet_12_colors[3,:],markersize=markersize0,label='mRNA')
    else:
        plt.plot(np.std(rna_total[:,int(tt*3600/step)],0),np.std(SampleSum[:,:,0],axis=1)[round(tt*3600/np.max(TimePoins)*TimePoins.shape[0])],
                  alpha=transparency[k], marker='o',linestyle = 'None',color=jet_12_colors[3,:],markersize=markersize0)
      
    k=k+1

plt.plot(np.array([0,Range]),np.array([0,Range]),linewidth=4,color='black')
plt.xlabel('Gillespie')
# plt.ylabel('VAN')
plt.title('Std')
plt.legend(fontsize=48,loc='best',handletextpad=0.2)
fig.set_size_inches(9, 8)
plt.savefig('GE_panelb21.svg', bbox_inches="tight", dpi=400)

#Protein
plt.figure(num=None,  dpi=400, edgecolor='k', linewidth=8)
fig, axes = plt.subplots(1,1)
Range=10
k=0
for tt in np.arange(0,T,NumTimePoints):
    if tt==0:
        plt.plot(np.std(prot_total[:,int(tt*3600/step)],0),np.std(SampleSum[:,:,1],axis=1)[round(tt*3600/np.max(TimePoins)*TimePoins.shape[0])],
                  alpha=transparency[k], marker='o',linestyle = 'None',color=jet_12_colors[12,:],markersize=markersize0,label='Protein')
    else:
        plt.plot(np.std(prot_total[:,int(tt*3600/step)],0),np.std(SampleSum[:,:,1],axis=1)[round(tt*3600/np.max(TimePoins)*TimePoins.shape[0])],
                  alpha=transparency[k], marker='o',linestyle = 'None',color=jet_12_colors[12,:],markersize=markersize0)     
    k=k+1
plt.plot(np.array([0,Range]),np.array([0,Range]),linewidth=4,color='black')
plt.xlabel('Gillespie')
# plt.ylabel('VAN')
plt.title('Std')
plt.legend(fontsize=48,loc='best',handletextpad=0.2)
fig.set_size_inches(9, 8)
plt.savefig('GE_panelb22.svg', bbox_inches="tight", dpi=400)

<table>
<td> 
<img src="https://github.com/jiadeyu0602/CheatSheet/raw/master/GE_panelb11.svg" width="190"/> <br>
</td> 
<td> 
<img src="https://github.com/jiadeyu0602/CheatSheet/raw/master/GE_panelb12.svg" width="200"/> <br>
<td> 
<img src="https://github.com/jiadeyu0602/CheatSheet/raw/master/GE_panelb21.svg" width="175"/> <br>
</td> 
<td> 
<img src="https://github.com/jiadeyu0602/CheatSheet/raw/master/GE_panelb22.svg" width="187"/> <br>
</td> 
</table>

### 5.3 The marginal distributions of species

In [ ]:
# The marginal distributions of the mRNA and protein at time point t=0.5 from the Gillespie simulation and the VAN. The inset shows the Hellinger distance between the two distributions.
####Distribution--------------
T2=180
T1=round(TimePoins[T2]/3600,3)

Range1=np.arange(0,8,0.7)
Range2=8

mRNA1=data1["arr_2"][:,int(T1*3600/step)]
mRNA2=data2["arr_5"][T2,:,0]
protein1=data1["arr_3"][:,int(T1*3600/step)]
protein2=data2["arr_5"][T2,:,1]

mRNA=[mRNA1,mRNA2]
print(np.shape(mRNA))
print(np.shape(mRNA1))
protein=[protein1,protein2]
df_mRNA1=pd.DataFrame(mRNA1)
df_mRNA2=pd.DataFrame(mRNA2)
pro_mRNA1=df_mRNA1.value_counts(normalize=True).sort_index()
pro_mRNA2=df_mRNA2.value_counts(normalize=True).sort_index() 
df_protein1=pd.DataFrame(protein1)
df_protein2=pd.DataFrame(protein2)
pro_protein1=df_protein1.value_counts(normalize=True).sort_index()
pro_protein2=df_protein2.value_counts(normalize=True).sort_index() 

HD_mRNA=round(np.sqrt(1-np.sum(np.sqrt(pro_mRNA1*pro_mRNA2))),3)
HD_protein=round(np.sqrt(1-np.sum(np.sqrt(pro_protein1*pro_protein2))),3)

plt.rc('font', size=36)
plt.rc('lines',linewidth=4)

plt.figure(num=None,  dpi=400, edgecolor='k', linewidth=4)
fig, axes = plt.subplots(1,1)
fig.tight_layout()
ax = plt.subplot(1,1, 1)
weights1 = np.ones_like(mRNA1)/float(len(mRNA1))
weights2 = np.ones_like(mRNA2)/float(len(mRNA2))
plt.hist(mRNA,bins=Range1,weights=[weights1,weights2],color=['darkgrey',jet_12_colors[3,:]],alpha=0.7,orientation='horizontal')
plt.legend(['Gillespie','VAN'],title='$D_{HD}=$'+str(HD_mRNA),fontsize=32)
plt.ylabel("mRNA")
plt.xlabel('Probability')
plt.title("$t=$"+str(T1))
fig.set_size_inches(9, 8)
plt.savefig('GE_panelc_mRNA_T'+str(T1)+'.svg', bbox_inches="tight", dpi=400)
plt.show()


plt.figure(num=None,  dpi=400, edgecolor='k', linewidth=4)
fig, axes = plt.subplots(1,1)
fig.tight_layout()
ax = plt.subplot(1,1, 1)
weights1 = np.ones_like(protein1)/float(len(protein1))
weights2 = np.ones_like(protein2)/float(len(protein2))
plt.hist(protein,bins=Range2,weights=[weights1,weights2],color=['darkgrey',jet_12_colors[12,:]],alpha=0.7,orientation='horizontal')
plt.legend(['Gillespie','VAN'],title='$D_{HD}=$'+str(HD_protein),fontsize=32)
plt.ylabel("Protein")
plt.xlabel('Probability')
plt.ylim(top=130)
plt.title("$t=$"+str(T1))
fig.set_size_inches(9,8)
plt.savefig('GE_panelc_protein_T'+str(T1)+'.svg', bbox_inches="tight", dpi=400)
plt.show()


<table>
<td> 
<img src="https://github.com/jiadeyu0602/CheatSheet/raw/master/GE_panelc_mRNA_T0.5.svg" width="200"/> <br>
</td> 
<td> 
<img src="https://github.com/jiadeyu0602/CheatSheet/raw/master/GE_panelc_protein_T0.5.svg" width="205"/> <br>
</td> 
</table>

### 5.4 The joint distribution of species

In [ ]:
#The joint distribution of the two proteins from the VAN at time point t=40, with the color bar for the probability values in the logarithmic scale.
###hist2d------------------------------------------------
newcmp='viridis'
plt.rc('font', size=36)

T=180
plt.figure(num=None,  dpi=400, edgecolor='k', linewidth=4)
fig, axes = plt.subplots(1,1)
fig.tight_layout()
ax = plt.subplot(1,1, 1,facecolor=[68/255,1/255,80/255])
SampleSum[T,0,1]=0
h=plt.hist2d(SampleSum[T,:,1],SampleSum[T,:,0],norm=mpl.colors.LogNorm(vmax=0.025),bins=[int(max(SampleSum[T,:,1]))+1,int(max(SampleSum[T,:,0]))+1],cmap=newcmp,density=True)
plt.colorbar(label='Probability')
plt.xlabel("Protein")
plt.ylabel("mRNA")
plt.title("$t=$0.5")
plt.xlim(0,80)
plt.ylim(0,6)
# plt.yticks([0,1])
# plt.yticks([0,10,20,30,40])
fig.set_size_inches(11,8)
# fig.set_size_inches(2,8)
plt.savefig('GE_paneld_T'+str(T)+'.svg', bbox_inches="tight", dpi=400)
plt.show()


<table>
<td> 
<img src="https://github.com/jiadeyu0602/CheatSheet/raw/master/GE_paneld_T180.svg" width="300"/> <br>
</td> 
</table>